In [1]:
#Import basic necessary datasets
import pandas as pd
import numpy as np

In [2]:
#Read the datasets and view the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
#filling null values
train.bmi.fillna(value=train.bmi.median(), axis=0, inplace=True)
test.bmi.fillna(value=test.bmi.median(), axis=0, inplace=True)

train.smoking_status.fillna(value='no information', axis=0, inplace=True)
test.smoking_status.fillna(value='no information', axis=0, inplace=True)

In [4]:
#grouping the data
train.age = np.digitize(train.age, bins=np.arange(start=train.age.min(),stop=train.age.max(),step=10))
test.age = np.digitize(test.age, bins=np.arange(start=test.age.min(),stop=test.age.max(),step=10))

train.avg_glucose_level = np.digitize(train.avg_glucose_level, bins=np.arange(start=train.avg_glucose_level.min(),stop=train.avg_glucose_level.max(),step=50))
test.avg_glucose_level = np.digitize(test.avg_glucose_level, bins=np.arange(start=test.avg_glucose_level.min(),stop=test.avg_glucose_level.max(),step=50))

train.bmi = np.digitize(train.bmi, bins=np.arange(start=train.bmi.min(),stop=train.bmi.max(),step=10))
test.bmi = np.digitize(test.bmi, bins=np.arange(start=test.bmi.min(),stop=test.bmi.max(),step=10))

In [5]:
train.bmi[train.bmi.isin([7,8,9])] = 6
test.bmi[test.bmi.isin([7,8,9])] = 6

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [6]:
#drop id column
train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)

In [7]:
#get dummy variables
train = pd.get_dummies(train, columns=['gender','age','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'],drop_first=True)
test = pd.get_dummies(test, columns=['gender','age','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'],drop_first=True)

In [48]:
#Take independent features into x and depenent feature into y
X = train.drop('stroke', axis=1)
y = train.stroke

In [9]:
from imblearn.over_sampling import SMOTE
from collections import Counter
sm = SMOTE()
X_res, y_res = sm.fit_sample(X, y)
Counter(y_res).items() #To print value counts of target classes

dict_items([(0, 42617), (1, 42617)])

In [49]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=123)
X_train = np.array(X_train)
X_test = np.array(X_test)

In [26]:
#import keras libraries
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers

In [181]:
#build a classifier function
def build_classifier():
    classifier = Sequential()  # initiate the model
    classifier.add(Dense(60, input_shape=(30,), activation='relu')) # input layer
    classifier.add(Dense(120, activation='relu'))                  # hidden layer
    classifier.add(Dense(50, activation='relu'))                  # hidden layer
    classifier.add(Dense(10, activation='relu'))                  # hidden layer
    #classifier.add(Dropout(0.25))
    classifier.add(Dense(1, activation = 'sigmoid')) #output layer
    #adamax = optimizers.Adamax(lr=0.1)
    classifier.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    return classifier

In [182]:
#create a KerasClassifier object and call build_classifier function
classifier = KerasClassifier(build_fn=build_classifier, batch_size=3000, nb_epoch=15, verbose=2)

In [183]:
from IPython.core.interactiveshell import InteractiveShell #To print multiple outputs
InteractiveShell.ast_node_interactivity = 'all'

In [184]:
accuracies = cross_val_score(classifier, X_train, y_train, cv=10)

Epoch 1/10
 - 6s - loss: 0.4156 - acc: 0.9599
Epoch 2/10
 - 0s - loss: 0.1266 - acc: 0.9821
Epoch 3/10
 - 0s - loss: 0.1061 - acc: 0.9821
Epoch 4/10
 - 0s - loss: 0.1011 - acc: 0.9821
Epoch 5/10
 - 0s - loss: 0.0924 - acc: 0.9821
Epoch 6/10
 - 0s - loss: 0.0889 - acc: 0.9821
Epoch 7/10
 - 0s - loss: 0.0859 - acc: 0.9821
Epoch 8/10
 - 0s - loss: 0.0835 - acc: 0.9821
Epoch 9/10
 - 0s - loss: 0.0813 - acc: 0.9821
Epoch 10/10
 - 0s - loss: 0.0795 - acc: 0.9821
Epoch 1/10
 - 6s - loss: 0.5313 - acc: 0.8931
Epoch 2/10
 - 0s - loss: 0.2028 - acc: 0.9821
Epoch 3/10
 - 0s - loss: 0.1006 - acc: 0.9821
Epoch 4/10
 - 0s - loss: 0.1019 - acc: 0.9821
Epoch 5/10
 - 0s - loss: 0.0920 - acc: 0.9821
Epoch 6/10
 - 0s - loss: 0.0867 - acc: 0.9821
Epoch 7/10
 - 0s - loss: 0.0841 - acc: 0.9821
Epoch 8/10
 - 0s - loss: 0.0817 - acc: 0.9821
Epoch 9/10
 - 0s - loss: 0.0798 - acc: 0.9821
Epoch 10/10
 - 0s - loss: 0.0783 - acc: 0.9821
Epoch 1/10
 - 6s - loss: 0.6348 - acc: 0.8551
Epoch 2/10
 - 0s - loss: 0.4216 

In [185]:
accuracies.mean()
accuracies.std()

0.98211405764801718

0.0012113937237098905

In [186]:
classifier.fit(X_train, y_train)

Epoch 1/10
 - 7s - loss: 0.5148 - acc: 0.9527
Epoch 2/10
 - 0s - loss: 0.1894 - acc: 0.9821
Epoch 3/10
 - 0s - loss: 0.1035 - acc: 0.9821
Epoch 4/10
 - 0s - loss: 0.1029 - acc: 0.9821
Epoch 5/10
 - 0s - loss: 0.0918 - acc: 0.9821
Epoch 6/10
 - 0s - loss: 0.0877 - acc: 0.9821
Epoch 7/10
 - 0s - loss: 0.0842 - acc: 0.9821
Epoch 8/10
 - 0s - loss: 0.0813 - acc: 0.9821
Epoch 9/10
 - 0s - loss: 0.0788 - acc: 0.9821
Epoch 10/10
 - 0s - loss: 0.0769 - acc: 0.9821


In [187]:
from sklearn import metrics
y_pred = classifier.predict_proba(X_test)
metrics.roc_auc_score(y_test,y_pred[:,1])

0.8307817287429089

In [188]:
test_pred = classifier.predict_proba(np.array(test))
test_pred.max()

0.99994248

In [189]:
sample['stroke'] = test_pred[:,1]
sample.to_csv('sample_submission.csv',index=False)
sample.head()

,id,stroke
0,36306,0.071771
1,61829,0.154766
2,14152,0.000504
3,12997,0.001908
4,40801,0.044137


In [190]:
sample.stroke.min()
sample.stroke.max()

5.7519585e-05

0.23404188